In [17]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import numpy as np
import base64
from PIL import Image
from io import BytesIO
from tensorflow.keras.models import load_model
import pyvjoy

In [3]:
sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data = {"steering_angle": str(steering_angle),
                             "throttle": str(throttle)}, skip_sid = True)

def process_image(image):
    return image[10:130:2, ::4, :]

In [3]:
model = load_model('f:\Pliki ściągnięte\mymodel1 (1).h5')

In [ ]:
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        
        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
#         print(image_array.shape)
        img = process_image(image_array)
#         print(img.shape)
        img_batch = np.expand_dims(img, axis=0)
#         print(img_batch.shape)

        steering_angle = model.predict(img_batch)
        throttle = 0.15
        if speed < 15:
            throttle = 0.8
        if speed > 17:
            throttle = -0.1

        send_control(steering_angle[0][0], throttle)
    else:
        sio.emit("manual", data = (), skip_sid = True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('',4567)), app)

(18408) wsgi starting up on http://0.0.0.0:4567
(18408) accepted ('127.0.0.1', 62100)
(18408) accepted ('127.0.0.1', 62102)
127.0.0.1 - - [23/Jan/2020 22:54:27] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 124.542634
127.0.0.1 - - [23/Jan/2020 22:56:26] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 175.554998


In [72]:
MAX_VJOY = 32767
NEUTRAL_VJOY = MAX_VJOY/2
j = pyvjoy.VJoyDevice(1)

def play_function(X,Y,Z,XRot):
    j.data.wAxisX = int(X * self.MAX_VJOY)
    j.data.wAxisY = int(Y * self.MAX_VJOY)
    j.data.wAxisZ = int(Z * self.MAX_VJOY)
    j.data.wAxisXRot = int(XRot * self.MAX_VJOY)
    j.update()
    

In [113]:
MAX_VJOY = 32767
NEUTRAL_VJOY = int(MAX_VJOY/2)
j = pyvjoy.VJoyDevice(1)
j.set_button(1,1)
j.set_button(1,0)
# j.set_axis(pyvjoy.HID_USAGE_X, MAX_VJOY)
# j.set_axis(pyvjoy.HID_USAGE_X, 32767)

j.data.wAxisX = NEUTRAL_VJOY 
j.data.wAxisY= MAX_VJOY

j.data.wAxisXRot = MAX_VJOY 
j.data.wAxisYRot = NEUTRAL_VJOY 
j.update()

1